In [1]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# Install Pipeline SDK - This only needs to be ran once in the enviroment. 
!pip3 install kfp --upgrade
!pip3 install tensorflow==1.14

## KubeFlow Pipelines Serving Component
In this notebook, we will demo:

* Saving a Keras model in a format compatible with TF Serving
* Creating a pipeline to serve a trained model within a KubeFlow cluster

Reference documentation:
* https://www.tensorflow.org/tfx/serving/architecture
* https://www.tensorflow.org/beta/guide/keras/saving_and_serializing
* https://www.kubeflow.org/docs/components/serving/tfserving_new/

### Setup


In [3]:
# Set your output and project. !!!Must Do before you can proceed!!!
PROJECT_NAME =  'Your-Gcp-Project-ID'                          #'Your-GCP-Project-ID'
MODEL_NAME = 'Model-Name'                                      # Model name matching TF_serve naming requirements       
EXPERIMENT_NAME = 'serving_component'
MODEL_VERSION = '1'                                            # A number representing the version model 
OUTPUT_BUCKET = 'gs://%s-serving-component' % PROJECT_NAME     # A GCS bucket for asset outputs
KUBEFLOW_DEPLOYER_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-kubeflow-deployer:fe639f41661d8e17fcda64ff8242127620b80ba0'
MODEL_PATH = '%s/%s' % (OUTPUT_BUCKET,MODEL_NAME)               
MODEL_VERSION_PATH = '%s/%s/%s' % (OUTPUT_BUCKET,MODEL_NAME,MODEL_VERSION)

In [4]:
!gsutil mb {OUTPUT_BUCKET}

Creating gs://chavoshi-dev-2-serving-component/...


In [5]:
#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()

try:
    experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except:
    experiment = client.create_experiment(EXPERIMENT_NAME)

### Load a Keras Model 
Loading a pretrained Keras model to use as an example. 

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.applications.NASNetMobile(input_shape=None,
                                   include_top=True,
                                   weights='imagenet',
                                   input_tensor=None,
                                   pooling=None,
                                   classes=1000)

W0813 22:25:44.771417 139829630277440 deprecation.py:506] From /opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Saved the Model for TF-Serve
Save the model using keras export_saved_model function. Note that specifically for TF-Serve the output directory should be structure as model_name/model_version/saved_model.

In [8]:
tf.keras.experimental.export_saved_model(model,MODEL_VERSION_PATH)

W0813 22:26:30.974738 139829630277440 deprecation.py:506] From /opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0813 22:26:30.992686 139829630277440 deprecation.py:506] From /opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0813 22:26:31.067510 139829630277440 deprecation.py:506] From /opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecat

### Create a pipeline using KFP TF-Serve component


In [9]:
def kubeflow_deploy_op():
    return dsl.ContainerOp(
        name = 'deploy',
        image = KUBEFLOW_DEPLOYER_IMAGE,
        arguments = [
            '--model-export-path', MODEL_PATH,
            '--server-name', MODEL_NAME,
        ]
    )

In [10]:
import kfp
import kfp.dsl as dsl
from kfp.gcp import use_gcp_secret

# The pipeline definition
@dsl.pipeline(
    name='Sample model deployer',
    description='Sample for deploying models using KFP model serving component'
)
def model_server():
    deploy = kubeflow_deploy_op().apply(use_gcp_secret('user-gcp-sa'))

In [11]:
pipeline_func = model_server
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'

import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [12]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#This link leads to the run information page. 
#Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working